Import necessary libraries

In [4]:
!pip install numpy 

  Using cached numpy-2.3.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.7 MB 2.4 MB/s eta 0:00:06
   -- ------------------------------------- 0.8/12.7 MB 2.0 MB/s eta 0:00:07
   --- ------------------------------------ 1.0/12.7 MB 1.7 MB/s eta 0:00:07
   ---- ----------------------------------- 1.3/12.7 MB 1.5 MB/s eta 0:00:08
   ---- ----------------------------------- 1.6/12.7 MB 1.5 MB/s eta 0:00:08
   ------ --------------------------------- 2.1/12.7 MB 1.4 MB/s eta 0:00:08
   ------- -------------------------------- 2.4/12.7 MB 1.5 MB/s eta 0:00:08
   -------- ------------------------------- 2.6/12.7 MB 1.5 MB/s eta 0:00:07
   --------- ------------------------------ 2.9/12.7 MB 1.4 MB/s eta 0:00:08
   --------- ------------------------------ 3.1/12.7 MB 1.4 MB/s eta 0:00:07
   ---------- ------

In [5]:
!pip install pandas

  Using cached pandas-2.3.0-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 1.9 MB/s eta 0:00:06
   --- ------------------------------------ 1.0/11.0 MB 2.3 MB/s eta 0:00:05
   ----- ---------------------------------- 1.6/11.0 MB 2.6 MB/s eta 0:00:04
   -------- ------------------------------- 2.4/11.0 MB 2.8 MB/s eta 0:00:04
   ---------- ----------------------------- 2.9/11.0 MB 2.6 MB/s eta 0:00:04
   ----------- ---------------------------- 3.1/11.0 MB 2.6 MB/s eta 0:00:03
   ------------ --------------------------- 3.4/11.0 MB 2.5 MB/s eta 0:00:04
   -------------- ------------------------- 3.9/11.0 MB 2.3 MB/s eta 0:00:04
   ---------------- --------------------

In [6]:
!pip install scikit-learn matplotlib seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   -- ------------------------------------- 0.8/10.7 MB 3.7 MB/s eta 0:00:03
   ------ --------------------------------- 1.8/10.7 MB 4.6 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/10.7 MB 4.7 MB/s eta 0:00:02
   ------------ --------------------------- 3.4/10.7 MB 4.7 MB/s eta 0:00:02
   ---------------- ----------------------- 4.5/10.7 MB 4.6 MB/s eta 0:00:02
   -------------------- ------------------- 5.5/10.7 MB 4.4 MB/s eta 0:00:02
   ------------------------ --------------- 6.6/10.7 MB 4.5 MB/s eta 0:00:01
   ---------------------------- ----------- 7.6/10.7 MB 4.4 MB/s

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.feature_selection import mutual_info_classif, f_classif
from sklearn.decomposition import PCA
import warnings

Load the dataset

In [8]:
df = pd.read_csv('heart.csv')

Display shape, info, describe and others

In [9]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [10]:
df.shape

(918, 12)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [12]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Age,918.0,53.510893,9.432617,28.0,47.00,54.0,60.0,77.0
RestingBP,918.0,132.396514,18.514154,0.0,120.00,130.0,140.0,200.0
Cholesterol,918.0,198.799564,109.384145,0.0,173.25,223.0,267.0,603.0
FastingBS,918.0,0.233115,0.423046,0.0,0.00,0.0,0.0,1.0
MaxHR,918.0,136.809368,25.460334,60.0,120.00,138.0,156.0,202.0
Oldpeak,918.0,0.887364,1.066570,-2.6,0.00,0.6,1.5,6.2
HeartDisease,918.0,0.553377,0.497414,0.0,0.00,1.0,1.0,1.0


In [13]:
df.nunique()

Age                50
Sex                 2
ChestPainType       4
RestingBP          67
Cholesterol       222
FastingBS           2
RestingECG          3
MaxHR             119
ExerciseAngina      2
Oldpeak            53
ST_Slope            3
HeartDisease        2
dtype: int64

In [14]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [15]:
df.isnull().sum().sum()

np.int64(0)

Identifying categorical and numerical columns

In [16]:
categorical_cols = df.select_dtypes(include='object').columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns